### CS559 - Homework #2

**Author**: Sid Bhatia

**Date**: September 18th, 2024

**Pledge**: I pledge my honor that I have abided by the Stevens Honor System.

**Professor**: Dr. In Suk Jang

#### 1. Naive Bayes Classification [40 pts]

Use the following code to generate the train data set. The code will generate a random data set with four features and classes.

```python
from sklearn import datasets
X, y = datasets.make_blobs(n_samples = 400, n_features = 5, centers = 4, cluster_std = 2, random_state = 100)
```

a. [5pts] Compute the prior probability of each class, $p(C_k), \; \forall \, k = 1, ..., 4$.

In [10]:
from sklearn import datasets
import numpy as np

X, y = datasets.make_blobs(n_samples = 400, n_features = 5, centers = 4, cluster_std = 2, random_state = 100)

unique_classes, counts = np.unique(y, return_counts=True)
prior_probabilities = counts / len(y)

print(prior_probabilities)

[0.25 0.25 0.25 0.25]


b. [10pts] Compute the likelihood $p(X \mid C_k), \; \forall \, 1, ..., 4.$

In [16]:
from scipy.stats import multivariate_normal

class_means = {}
class_covariances = {}
class_likelihoods = {}

for cls in unique_classes:

    X_cls = X[y == cls]
    
    mean_cls = np.mean(X_cls, axis=0)
    class_means[cls] = mean_cls
    
    cov_cls = np.cov(X_cls, rowvar=False)
    class_covariances[cls] = cov_cls
    
    likelihood_cls = multivariate_normal.pdf(X, mean=mean_cls, cov=cov_cls)
    class_likelihoods[cls] = likelihood_cls

    # print(f"\nClass {cls + 1} Mean Vector:\n{mean_cls}")
    # print(f"\nClass {cls + 1} Covariance Matrix:\n{cov_cls}")
    print(f"\nLikelihoods p(X | C_{cls + 1}):\n{likelihood_cls}")


Likelihoods p(X | C_1):
[5.09197236e-19 9.50612419e-19 1.41933448e-04 1.88275100e-16
 2.38657046e-25 7.29170761e-39 2.43027345e-20 8.17664284e-31
 3.26195346e-33 6.85670933e-24 5.78523664e-06 3.60430084e-22
 7.63069571e-23 1.72159590e-18 2.25937526e-05 2.30055471e-19
 1.16451071e-20 2.48679170e-21 5.07156819e-35 7.60305205e-17
 3.33809017e-24 3.33218481e-32 1.81842343e-31 1.14384989e-19
 1.47678746e-30 4.44560186e-19 1.39904274e-31 5.78745571e-06
 1.13347716e-21 3.92274007e-05 1.70100195e-21 9.77184782e-19
 8.87339315e-05 9.08519999e-06 7.59823498e-25 1.70267266e-07
 7.42463459e-29 9.07741727e-21 5.30840745e-34 2.27405539e-22
 6.42801233e-32 2.52838036e-14 2.41139892e-21 6.48094751e-05
 2.08289315e-04 3.38862891e-16 4.93226655e-28 4.55649828e-24
 2.57241829e-24 3.90905967e-32 3.81471713e-20 3.57734601e-28
 3.05998922e-26 1.27978072e-29 1.55396506e-17 1.69126268e-33
 6.81796607e-05 4.80211010e-33 6.03954106e-17 5.61737488e-20
 2.80132538e-20 9.51046379e-07 4.78818295e-17 3.89533155e-33

c. [15pts] Compute the posterior probability of each point $p(C_k \mid X), \; \forall \, k = 1, ..., 4$. Assign the class ID to each point.

In [28]:
num_classes = len(unique_classes)
num_samples = X.shape[0]

posterior_probabilities = np.zeros((num_samples, num_classes))
for idx, cls in enumerate(unique_classes):
    prior = prior_probabilities[cls]
    likelihood = class_likelihoods[cls]
    posterior_probabilities[:, idx] = likelihood * prior

posterior_probabilities_sum = np.sum(posterior_probabilities, axis=1, keepdims=True)
posterior_probabilities_normalized = posterior_probabilities / posterior_probabilities_sum

predicted_classes = np.argmax(posterior_probabilities_normalized, axis=1)
predicted_class_labels = unique_classes[predicted_classes]

for i in range(5):
    print(f"\nData Point {i + 1}:")
    print(f"True Class: {y[i] + 1}")
    for idx, cls in enumerate(unique_classes):
        print(f"p(C_{cls + 1} | x_{i + 1}) = {posterior_probabilities_normalized[i, idx]:.4f}")
    print(f"Assigned Class: {predicted_class_labels[i] + 1}")


Data Point 1:
True Class: 4
p(C_1 | x_1) = 0.0000
p(C_2 | x_1) = 0.0000
p(C_3 | x_1) = 0.0000
p(C_4 | x_1) = 1.0000
Assigned Class: 4

Data Point 2:
True Class: 3
p(C_1 | x_2) = 0.0000
p(C_2 | x_2) = 0.0000
p(C_3 | x_2) = 1.0000
p(C_4 | x_2) = 0.0000
Assigned Class: 3

Data Point 3:
True Class: 1
p(C_1 | x_3) = 1.0000
p(C_2 | x_3) = 0.0000
p(C_3 | x_3) = 0.0000
p(C_4 | x_3) = 0.0000
Assigned Class: 1

Data Point 4:
True Class: 4
p(C_1 | x_4) = 0.0000
p(C_2 | x_4) = 0.0000
p(C_3 | x_4) = 0.0000
p(C_4 | x_4) = 1.0000
Assigned Class: 4

Data Point 5:
True Class: 4
p(C_1 | x_5) = 0.0000
p(C_2 | x_5) = 0.0000
p(C_3 | x_5) = 0.0000
p(C_4 | x_5) = 1.0000
Assigned Class: 4


d. [5pts] Construct the confusion matrix to show the classification rate using `sklearn.metrics.confusion_matrix`. The confusion matrix should visualize and summarize the performance of a classification algorithm.

In [31]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y, predicted_class_labels)

print(conf_matrix)

[[100   0   0   0]
 [  0 100   0   0]
 [  0   0 100   0]
 [  0   0   0 100]]


e. [5pts] Classify the target using `sklearn.native_bayes.GaussianNB`. Report the accuracy of the model.

In [33]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

gnb = GaussianNB()

gnb.fit(X, y)

y_pred = gnb.predict(X)

accuracy = accuracy_score(y, y_pred)

print(accuracy)

1.0


#### 2. Perceptron [30 pts]

a. [15 pts] In the lecture slide, the methods needed for the perceptron algorithm are provided: `step(X)` and `perceptron_predict(w, X)`. Write a method (`Perceptron_fit(w, X, y, learning_rate, iteration`) that fits the data and returns `w`.

In [36]:
def step(X):
    return 1 if X > 0 else -1

def perceptron_predict(w, X):
    return step(np.dot(np.append(1, X), w))

def perceptron_fit(w, X, y, learning_rate, iterations):

    X_bias = np.c_[np.ones(X.shape[0]), X]
    
    for j in range(iterations):

        for k in range(len(y)):

            h = step(np.dot(X_bias[k], w))
            
            if h != y[k]:

                error = y[k] - h
                
                w += learning_rate * error * X_bias[k]
                
    return w